In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
import scipy.sparse
import random


train_matrix_path = '../Data/train_matrix.npz'
test_matrix_path = '../Data/test_matrix.npz'
unique_ids_path = '../Data/unique.npz'

# TODO
train_matrix_coo = scipy.sparse.load_npz(train_matrix_path)
test_matrix_coo = scipy.sparse.load_npz(test_matrix_path)

unique_user_ids = np.load(unique_ids_path, allow_pickle=True)['users']
unique_item_ids = np.load(unique_ids_path, allow_pickle=True)['items']

train_matrix_csr = train_matrix_coo.tocsr()
train_matrix_csc = train_matrix_coo.tocsc()


print(f"Train matrix shape (CSR): {train_matrix_csr.shape}")
print(f"Train matrix shape (CSC): {train_matrix_csc.shape}")
print(f"Test matrix shape (COO): {test_matrix_coo.shape}")

Train matrix shape (CSR): (64078, 144804)
Train matrix shape (CSC): (64078, 144804)
Test matrix shape (COO): (64078, 144804)


<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام اول
    </font>
  </h2>
</div>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazirmatn; font-size:medium">
<font face="vazirmatn" size=3>
  برای ساخت ماتریس‌های ویژگی‌های پنهان کاربر (P) و ماتریس ویژگی پنهان آیتم (Q) به تعداد کاربران و آیتم‌ها نیاز داریم. به ترتیب در متغیرهای <code>num_users</code> و <code>num_items</code> تعداد کاربران و آیتم‌ها را ذخیره کنید. برای این‌کار می‌توانید از ابعاد ماتریس <code>train_matrix_csr</code> کمک بگیرید.
</font>
</p>


In [ ]:
# TODO
num_users, num_items = train_matrix_csr.shape

print(f"Number of unique users: {num_users}")
print(f"Number of unique items: {num_items}")

Number of unique users: 64078
Number of unique items: 144804


In [ ]:
# TODO
k = 5
lambda_reg = 10
num_epochs = 10
log_step = 1

In [ ]:
# TODO
P = np.random.rand(num_users, k) * 0.001
Q = np.random.rand(num_items, k) * 0.001

user_bias = np.zeros(num_users)
item_bias = np.zeros(num_items)

global_average = np.mean(train_matrix_csr.data)

In [ ]:
global_average

7.629935528238631

In [ ]:
def solve_user_features(user_idx, train_matrix_csr, P, Q, user_bias, item_bias, global_average, lambda_reg, k):
    """
    Solves for the latent features and bias of a single user using Least Squares.

    Args:
        user_idx (int): The mapped index of the user.
        train_matrix_csr (csr_matrix): The training interaction matrix (CSR format).
        P (np.ndarray): Current user latent feature matrix (num_users, k).
        Q (np.ndarray): Current item latent feature matrix (num_items, k).
        user_bias (np.ndarray): Current user bias vector (num_users,).
        item_bias (np.ndarray): Current item bias vector (num_items,).
        global_average (float): The global average rating.
        lambda_reg (float): The regularization parameter.

    Returns:
        tuple: A tuple containing:
            - updated_user_bias (float): The updated bias for the user.
            - updated_user_features (np.ndarray): The updated latent features for the user (k,).
    """
    # TODO
    #--step 1:
    #  Get user's observed interactions fron train_matrix_csr
    user_row = train_matrix_csr.getrow(user_idx)
    rated_item_indices = user_row.indices # Mapped indices of items rated by this user
    user_ratings = user_row.data         # Actual ratings given by this user

    num_rated_items = len(rated_item_indices)

    if num_rated_items == 0:
        return user_bias[user_idx], P[user_idx, :]

    #--step 2:
    #  Get the latent features and biases for the items rated by this user
    item_features_subset = Q[rated_item_indices, :] # Shape (num_rated_items, k)
    item_biases_subset = item_bias[rated_item_indices] # Shape (num_rated_items,)


    #--step 3:
    #  Formulate the Least Squares problem: A_u * x_u = b_u

    # x_u is the vector of unknowns: [user_bias[user_idx], P[user_idx, 0], ..., P[user_idx, k-1]] (shape k+1)
    # A_u has shape (num_rated_items, k+1)
    # b_u has shape (num_rated_items,)

    #--step 3.1 and 3.2:
    # Construct A_u: First column is ones (for the bias term), remaining columns are item features
    A_u = np.hstack((np.ones((num_rated_items, 1)), item_features_subset)) # Shape (num_rated_items, k+1)

    #--step 3.3:
    # Construct b_u: Observed ratings minus global average and item biases
    b_u = user_ratings - global_average - item_biases_subset # Shape (num_rated_items,)

    # Formulate the normal equations: (A_u^T * A_u + lambda * I) * x_u = A_u^T * b_u
    ATA = A_u.T @ A_u # Shape (k+1, k+1)
    ATb = A_u.T @ b_u # Shape (k+1,)


    #--step 3.4:
    # Add regularization term (lambda * I)
    # Create an identity matrix of size k+1
    regularization_matrix = np.eye(k + 1)
    regularization_matrix[0, 0] = 0
    # Create ATA_regularized matrix
    ATA_regularized = ATA + lambda_reg * regularization_matrix


    #--step 4:
    # Solve the linear system
    # Use np.linalg.solve(matrix, vector)
    solution_vector = np.linalg.solve(ATA_regularized, ATb) # Shape (k+1,)

    #--step 5:
    # Extract the updated bias and latent features from the solution vector
    updated_user_bias = solution_vector[0]
    updated_user_features = solution_vector[1:] # Shape (k,)

    return updated_user_bias, updated_user_features

In [ ]:
def solve_item_features(item_idx, train_matrix_csc, P, Q, user_bias, item_bias, global_average, lambda_reg, k):
    """
    Solves for the latent features and bias of a single item using Least Squares.

    Args:
        item_idx (int): The mapped index of the item.
        train_matrix_csc (csc_matrix): The training interaction matrix (CSC format).
        P (np.ndarray): Current user latent feature matrix (num_users, k).
        Q (np.ndarray): Current item latent feature matrix (num_items, k).
        user_bias (np.ndarray): Current user bias vector (num_users,).
        item_bias (np.ndarray): Current item bias vector (num_items,).
        global_average (float): The global average rating.
        lambda_reg (float): The regularization parameter.

    Returns:
        tuple: A tuple containing:
            - updated_item_bias (float): The updated bias for the item.
            - updated_item_features (np.ndarray): The updated latent features for the item (k,).
    """
    # TODO

    #--step 1:
    # Get the column for this item from the training matrix
    item_col = train_matrix_csc.getcol(item_idx)
    rating_user_indices = item_col.indices # Mapped indices of users who rated this item
    item_ratings = item_col.data         # Actual ratings given to this item

    num_rating_users = len(rating_user_indices)

    if num_rating_users == 0:
        return item_bias[item_idx], Q[item_idx, :]

    #--step 2:
    # Get the latent features and biases for the users who rated this item
    user_features_subset = P[rating_user_indices, :] # Shape (num_rating_users, k)
    user_biases_subset = user_bias[rating_user_indices] # Shape (num_rating_users,)

    #--step 3:
    # Formulate the Least Squares problem: A_i * x_i = b_i
    # x_i is the vector of unknowns: [item_bias[item_idx], Q[item_idx, 0], ..., Q[item_idx, k-1]] (shape k+1)
    # A_i has shape (num_rating_users, k+1)
    # b_i has shape (num_rating_users,)

    # Construct A_i: First column is ones (for the bias term), remaining columns are user features
    A_i = np.hstack((np.ones((num_rating_users, 1)), user_features_subset)) # Shape (num_rating_users, k+1)

    # Construct b_i: Observed ratings minus global average and user biases
    b_i = item_ratings - global_average - user_biases_subset # Shape (num_rating_users,)

    # Formulate the normal equations: (A_i^T * A_i + lambda * I) * x_i = A_i^T * b_i
    ATA = A_i.T @ A_i # Shape (k+1, k+1)
    ATb = A_i.T @ b_i # Shape (k+1,)

    # Add regularization term (lambda * I)
    regularization_matrix = np.eye(k + 1)
    # If not regularizing bias: regularization_matrix[0, 0] = 0
    regularization_matrix[0, 0] = 0
    ATA_regularized = ATA + lambda_reg * regularization_matrix

    #--step 4:
    # Solve the linear system
    solution_vector = np.linalg.solve(ATA_regularized, ATb) # Shape (k+1,)

    #--step 5:
    # Extract the updated bias and latent features from the solution vector
    updated_item_bias = solution_vector[0]
    updated_item_features = solution_vector[1:] # Shape (k,)

    return updated_item_bias, updated_item_features

In [ ]:
def predict(user_idx, item_idx, P, Q, user_bias, item_bias, global_average):
    """
    Predicts the rating for a given user and item using the trained ALS model.

    Args:
        user_idx (int): Mapped index of the user.
        item_idx (int): Mapped index of the item.
        P (np.ndarray): Trained user latent feature matrix (num_users, k).
        Q (np.ndarray): Trained item latent feature matrix (num_items, k).
        user_bias (np.ndarray): Trained user bias vector (num_users,).
        item_bias (np.ndarray): Trained item bias vector (num_items,).
        global_average (float): Calculated global average rating.

    Returns:
        float: The predicted rating.
    """
    try:
        predicted_rating = global_average + user_bias[user_idx] + item_bias[item_idx] + np.dot(P[user_idx, :], Q[item_idx, :])
        return predicted_rating

    except IndexError:
        print("\nCould not find mapped index for example prediction. Ensure original IDs exist in unique IDs.")
        return global_average

mapped_user_0_idx = 0
mapped_item_2_idx = 2
predicted_rating_example = predict(mapped_user_0_idx, mapped_item_2_idx, P, Q, user_bias, item_bias, global_average)
print(f"\nPredicted rating for mapped user {mapped_user_0_idx} and mapped item {mapped_item_2_idx}: {predicted_rating_example:.4f}")


Predicted rating for mapped user 0 and mapped item 2: 7.6299


In [ ]:
def calculate_rmse(P, Q, user_bias, item_bias, global_average, matrix_coo):
    """
    Calculates the Root Mean Squared Error (RMSE) on a sparse matrix using the trained ALS model.

    Args:
        P (np.ndarray): Trained user latent feature matrix.
        Q (np.ndarray): Trained item latent feature matrix.
        user_bias (np.ndarray): Trained user bias vector.
        item_bias (np.ndarray): Trained item bias vector.
        global_average (float): Calculated global average rating.
        matrix_csr (csr_matrix): The sparse matrix to evaluate on (e.g., test set).

    Returns:
        float: The calculated RMSE.
    """
    rows = matrix_coo.row
    cols = matrix_coo.col
    data = matrix_coo.data
    num_samples = len(data)

    if num_samples == 0:
        return 0.0

    sse = 0
    for idx in range(num_samples):
        u = rows[idx]
        i = cols[idx]
        r_ui = data[idx]

        predicted_r_ui = predict(u, i, P, Q, user_bias, item_bias, global_average)

        sse += (r_ui - predicted_r_ui)**2

    rmse = np.sqrt(sse / num_samples)
    return rmse

In [ ]:
print("\nStarting ALS training...")

for iteration in range(num_epochs):
    # --- Step 1: Solve for User Features (Fix Q and item_bias) ---
    # Iterate through each user
    for u in range(num_users):
        # TODO
        updated_b_u, updated_P_u = solve_user_features(u, train_matrix_csr, P, Q, user_bias, item_bias, global_average, lambda_reg, k)
        user_bias[u] = updated_b_u
        P[u, :] = updated_P_u

    # --- Step 2: Solve for Item Features (Fix P and user_bias) ---
    # Iterate through each item
    for i in range(num_items):
        # TODO
        updated_b_i, updated_Q_i = solve_item_features(i, train_matrix_csc, P, Q, user_bias, item_bias, global_average, lambda_reg, k)
        item_bias[i] = updated_b_i
        Q[i, :] = updated_Q_i

    if (iteration + 1) % log_step == 0:
        train_rmse = calculate_rmse(P, Q, user_bias, item_bias, global_average, train_matrix_coo)
        test_rmse = calculate_rmse(P, Q, user_bias, item_bias, global_average, test_matrix_coo)
        print(f"Iteration {iteration+1}/{num_epochs}, Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")


print("\nTraining finished.")


Starting ALS training...
Iteration 1/10, Train RMSE: 1.0124, Test RMSE: 1.7459
Iteration 2/10, Train RMSE: 0.9615, Test RMSE: 1.7650


KeyboardInterrupt: 

In [ ]:
mapped_user_0_idx = 0
mapped_item_2_idx = 2
predicted_rating_example = predict(mapped_user_0_idx, mapped_item_2_idx, P, Q, user_bias, item_bias, global_average)
print(f"\nPredicted rating for mapped user {mapped_user_0_idx} and mapped item {mapped_item_2_idx}: {predicted_rating_example:.4f}")


Predicted rating for mapped user 0 and mapped item 2: 5.6608


In [ ]:
def get_recommendations(original_user_id, n, P, Q, user_bias, item_bias, global_average, train_matrix_csr, unique_user_ids, unique_item_ids):
    """
    Generates top N recommendations for a given user using the trained ALS model.

    Args:
        original_user_id: The original ID of the user.
        n (int): The number of recommendations to generate.
        P (np.ndarray): Trained user latent feature matrix (num_users, k).
        Q (np.ndarray): Trained item latent feature matrix (num_items, k).
        user_bias (np.ndarray): Trained user bias vector (num_users,).
        item_bias (np.ndarray): Trained item bias vector (num_items,).
        global_average (float): Calculated global average rating.
        train_matrix_csr (csr_matrix): The sparse training interaction matrix.
        unique_user_ids (np.ndarray): Array mapping mapped user indices to original IDs.
        unique_item_ids (np.ndarray): Array mapping mapped item indices to original IDs.

    Returns:
        list: A list of tuples (original_item_id, predicted_rating) for the top N recommendations.
              Returns an empty list if the user ID is not found.
    """
    try:
        user_idx = np.where(unique_user_ids == original_user_id)[0][0]
    except IndexError:
        print(f"Warning: Original user ID {original_user_id} not found in unique user IDs.")
        return []

    num_items = Q.shape[0]
    all_item_indices = np.arange(num_items)

    user_row_sparse = train_matrix_csr.getrow(user_idx)
    rated_item_indices = user_row_sparse.indices


    unrated_item_indices = np.setdiff1d(all_item_indices, rated_item_indices)


    user_latent_features = P[user_idx, :]
    unrated_item_latent_features = Q[unrated_item_indices, :]
    unrated_item_biases = item_bias[unrated_item_indices]


    latent_predictions = np.dot(user_latent_features, unrated_item_latent_features.T)

    predicted_ratings_for_user = global_average + user_bias[user_idx] + unrated_item_biases + latent_predictions

    top_n_indices_in_unrated = np.argsort(predicted_ratings_for_user)[::-1][:n]

    recommended_mapped_item_indices = unrated_item_indices[top_n_indices_in_unrated]

    recommended_items_with_ratings = [(unique_item_ids[item_idx], predicted_ratings_for_user[i])
                                      for i, item_idx in enumerate(top_n_indices_in_unrated)]

    return recommended_items_with_ratings

In [ ]:
original_user_id_to_recommend = 8
top_n_recommendations = 5

recommendations = get_recommendations(original_user_id_to_recommend, top_n_recommendations, P, Q, user_bias, item_bias, global_average, train_matrix_csr, unique_user_ids, unique_item_ids)

if recommendations:
    print(f"\nTop {top_n_recommendations} recommendations for original user ID {original_user_id_to_recommend}:")
    for item_id, predicted_rating in recommendations:
        print(f"  Item ID: {item_id}, Predicted Rating: {predicted_rating:.4f}")


Top 5 recommendations for original user ID 8:
  Item ID: 1403332258, Predicted Rating: 5.6608
  Item ID: 0743230213, Predicted Rating: 5.7960
  Item ID: 0064471101, Predicted Rating: 5.6608
  Item ID: 0253216141, Predicted Rating: 5.7078
  Item ID: 0740723235, Predicted Rating: 6.1188


In [ ]:
import pandas as pd

eval_users_idx = [73,  75,  78,  81,  82,  83,  85,  86,  87,  88,  91,  92,  97,
        99, 102, 107, 109, 110, 114, 125]
eval_top_n_recommendations = 10
eval_recommendations = [get_recommendations(user_idx, eval_top_n_recommendations, P, Q, user_bias, item_bias, global_average, train_matrix_csr, unique_user_ids, unique_item_ids)
                        for user_idx in eval_users_idx]